In [6]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from numpy import cos, sin

init_notebook_mode(connected=True)

In [7]:
### 4.5.a)
from numpy import array, pi, cross, zeros
from orbitTools import UTC_time, Geodetic_To_ECEF, siderealTime, getOrbitalElements
import numpy as np

from orbitTools import eccentricAnomaly, ECEF_To_Geodetic 
from orbitTools import PERI_C_ECI, UTC_time, TrueAnomaly, ellipsoidPlot

DU = 6378.137;  #km
TU = 806.80415; #sec
VU = DU/TU;

# Earth physical constants from Vallado
muearth = 398600.4418;              # [km^3/sec^2]
reearth =   DU;               # [km] mean equatorial radius
rotEarthRad = 0.0000729211585530;  # [rad/sec]
rotEarthDay = 1.0027379093;  #rev/day
rotEarthDay = rotEarthRad*3600*24/(2*pi);
e2earth =      0.006694385000;      # oblate eccentricity squared

# Greenwich Sidereal Time at 0000h 1 January 2012 UTC
# from US Naval Observatory website
gst2012start = 6.6706801; # [sidereal hours] from vernal equinox

#convert to rad
gst2012start = gst2012start * 2*pi/24;    #[rad]

#tracking station information
track_lon = -(110 + 52/60 + 42/3600);   #deg
track_lat = 31 + 40/60 + 52/3600;       #deg
track_alt = 2.577;                      #km
track_UTC_offset = 7;       #hours behind UTC

track_lon = track_lon *pi/180;          #convert to rad
track_lat = track_lat *pi/180;          #convert to rad


#data collected from tracking station
time_obs = UTC_time(9,4,2012,18,54,0,track_UTC_offset);
rho = 39899.01730557;               #km
Az = 89.78381848      *pi/180;      #rad
El = 63.65779556      *pi/180;      #rad
rho_dot = 0.59337565;               #km/s
Az_dot = -0.00630053  *pi/180;      #rad/s
El_dot = 0.00299054   *pi/180;      #rad/s

# #data for checking purposes.  CHECK WORKS!!!
# rho = 35100; #[km]
# Az = 180*pi/180; # [rad]
# El = 55*pi/180; # [rad]
# rho_dot = 0; # [km/sec]
# Az_dot = 0*pi/180; # [rad/sec]
# El_dot = 0*pi/180; # [rad/sec]

#find R and V from these measurements in SEZ frame
a_R_s__sez = array([-rho*cos(Az)*cos(El),
               rho*sin(Az)*cos(El), rho*sin(El)]);
           
sez_V_s__sez = array(
    [-rho_dot*cos(Az)*cos(El) + rho*sin(Az)*Az_dot*cos(El) + rho*cos(Az)*sin(El)*El_dot,
      rho_dot*sin(Az)*cos(El) + rho*cos(Az)*Az_dot*cos(El) - rho*sin(Az)*sin(El)*El_dot,
      rho_dot*sin(El) + rho*cos(El)*El_dot])

#Express ground station in ECEF

o_R_a__ecef = Geodetic_To_ECEF(track_lat, track_lon, track_alt);

ecef__C__sez = array(
    [[sin(track_lat)*cos(track_lon), -sin(track_lon), cos(track_lat)*cos(track_lon)],
     [sin(track_lat)*sin(track_lon),  cos(track_lon), cos(track_lat)*sin(track_lon)],
     [-cos(track_lat),                 0,              sin(track_lat)]])

eci_omega_ecef__eci = array([0, 0, rotEarthRad])

# Find angle of Greenwich meridian from vernal equinox [0, 2*pi)
# theta_g = mod(gst2012start + rotEarthDay*(time_obs - 1)*2*pi,2*pi);
theta_g = siderealTime(time_obs);

# Convert ECI coordinates to ECEF coordinates
eci__C__ecef = array(
    [[cos(theta_g), -sin(theta_g), 0], 
     [sin(theta_g), cos(theta_g), 0], 
     [0, 0, 1]]);

# Calculate inertial R and V
o_R_s__eci = eci__C__ecef.dot(o_R_a__ecef + ecef__C__sez.dot(a_R_s__sez))
eci_V_s__eci = np.linalg.multi_dot([eci__C__ecef,ecef__C__sez,sez_V_s__sez]) + cross(eci_omega_ecef__eci,o_R_s__eci)



In [8]:
## 4.5.b)
#convert to canonical units
r = o_R_s__eci/DU;
v = eci_V_s__eci/VU;

[a, ecc, inc, raan, aop, nu0, meanmotion, M0] = getOrbitalElements(r,v);

#convert back from canonical units
a = a*DU;
meanmotion = meanmotion* (1/TU) * (1/(2*pi)) * 60*60*24; #rev/day

print("a = {0} km".format(a))
print("e = {0} ".format(ecc))
print("i = {0} deg".format(inc*180/pi))
print("raan = {0} deg".format(raan*180/pi))
print("aop = {0} deg".format(aop*180/pi))
print("nu0 = {0} deg".format(nu0*180/pi))
print("n = {0} rev/day".format(meanmotion))
print("M0 = {0} deg".format(M0*180/pi))

a = 42161.74437864817 km
e = 0.2674328876805949 
i = 63.939587411184625 deg
raan = 126.88149100866455 deg
aop = 270.2559968301256 deg
nu0 = 122.42515770419696 deg
n = 1.0028330989932042 rev/day
M0 = 93.73671524704686 deg


In [9]:
#%% 4.5.c)

print('This orbit is: direct, elliptical, inclined, geosynchronous')

#%% 4.5.d)

#% path(path,'Hw3')
orbitalElements=dict()
orbitalElements['a'] = a;
orbitalElements['e'] = ecc;
orbitalElements['i'] = inc;
orbitalElements['raan'] = raan;
orbitalElements['omega'] = aop;
orbitalElements['nu0'] = nu0;
orbitalElements['n'] = meanmotion;
orbitalElements['M0'] = M0;
orbitalElements['t0'] = time_obs;

startTime = UTC_time(1,5,2012,0,0,0,0);
stopTime = startTime + 1;

# plotOrbit(orbitalElements, startTime, stopTime, 1)

This orbit is: direct, elliptical, inclined, geosynchronous


In [11]:
from numpy import mod, pi
# plotOrbit(orbitalElements, startTime, stopTime, figNum):
"""#year is assumed to be 2012
#units on elements are assumed to be km, s, rad
#n is assumed to be rev/day
#times are assumed in UTC day + fraction"""

a = orbitalElements['a'];
ecc = orbitalElements['e'];
inc = orbitalElements['i'];
RAAN = orbitalElements['raan'];
omega = orbitalElements['omega'];
nu0 = orbitalElements['nu0'];
n = orbitalElements['n'];
M0 = orbitalElements['M0'];
t0 = orbitalElements['t0'];

if (ecc == 1):
    raise('not designed for parabolic orbits yet')
else:
    p = a*(1-ecc**2);

# Setup simulation time vector
# t = np.arange(startTime,stopTime,(1/(24*60*60))); 
t = np.arange(startTime,stopTime,(1/(24))); 
tLen = len(t)

# # Calculate orbital parameters
# n = (meanmotion*2.0*pi)/(24*60*60);        # [rad/sec]
# p = ((muearth/(n*n))**(1.0/3.0))*(1.0-ecc*ecc); # [km]

eci_C_peri = PERI_C_ECI(RAAN,inc,omega).T;             
# Notation note for eci_C_peri: This transforms from perifocal coordinates
# to ECI coordinates according to r_eci = eci_C_peri * r_peri
   
# Set aside space for data
# Nx3 matrices store eci or ecef positions at N datapoints.  Each row
# is one timestep, and columns 1, 2, and 3 hold the three components.
O_r_S__eci  = zeros((tLen,3)); # [km]  Sat position in ECI coordinates
O_r_S__ecef = zeros((tLen,3)); # [km]  Sat position in ECEF coordinates
lat_S       = zeros((tLen,1)); # [rad] Satellite geodetic latitude
lon_S       = zeros((tLen,1)); # [rad] Satellite longitude
he_S        = zeros((tLen,1)); # [km]  Satellite height above ellipsoid

# Simulate orbit by stepping through time vector
for ix in range(tLen):
    UTC = t[ix];

    # Find mean anomaly and put in range [0, 2*pi)
    MeanAnom = mod(n*(UTC - t0)*2*pi + M0,2*pi);
    
    # Find eccentric anomaly
    EccAnom = eccentricAnomaly([MeanAnom],ecc,1e-10);
     
    # Find position of satellite in perifocal coordinates
    TrueAnom = TrueAnomaly(EccAnom, MeanAnom, ecc);
    r = p/(1+ecc*cos(TrueAnom));
    
    O_r_S__peri = array([r*cos(TrueAnom), r*sin(TrueAnom), 0])
    
    # Convert perifocal coordinates to ECI coordinates
    O_r_S__eci[ix,:] = eci_C_peri.dot(O_r_S__peri)   # O_r_S__eci(i,:) is a 1x3 row matrix
   
    # Find angle of Greenwich meridian from vernal equinox [0, 2*pi)
    theta_g = mod(gst2012start + rotEarthDay*(UTC - 1)*2*pi,2*pi);
#     theta_g = gst2012start + 0.0657098244*floor(UTC) + 1.00273791*(UTC - floor(UTC))*24;
 
    # Convert ECI coordinates to ECEF coordinates
    ecef_C_eci = array([[cos(theta_g), sin(theta_g), 0], 
                        [-sin(theta_g), cos(theta_g), 0],
                        [0, 0, 1]])
    
    O_r_S__ecef[ix,:] = ecef_C_eci.dot(O_r_S__eci[ix,:].T);

    # Find geodetic latitude, longitude, and height above ellipsoid    
    [lat_S[ix], lon_S[ix], he_S[ix]] = ECEF_To_Geodetic(O_r_S__ecef[ix,0], O_r_S__ecef[ix,1], O_r_S__ecef[ix,2]);
  

In [12]:
# from plotly.graph_objs import Mesh3d

# from numpy import sin, cos, pi

layout = go.Layout(
    title='Inertial Observer View',
    width=800,
    height=800,
    scene=dict(
    aspectmode="cube",
    xaxis=dict(
        range=[-50000, 50000],
        dtick=10000,
        title='X [km]'
    ),
    yaxis=dict(
        range=[-50000, 50000],
        dtick=10000,
        title='Y [km]',
    ),
    zaxis=dict(
        range=[-50000, 50000],
        dtick=10000,
        title='Z [km]'
    ),)
)

data = [
    ellipsoidPlot(reearth),

    go.Scatter3d(x=O_r_S__eci[:,0],y=O_r_S__eci[:,1],z=O_r_S__eci[:,2],
                 name = 'orbit?',
                 marker=dict(
                     size=0,
                      color=0),
                 line = dict(color = ('red'), 
                             width = 1)
                )]


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [13]:
# from plotly.graph_objs import Mesh3d

# from numpy import sin, cos, pi

layout = go.Layout(
    title='Earth-Fixed Observer View',
    width=800,
    height=800,
    scene=dict(
    aspectmode="cube",
    xaxis=dict(
        range=[-50000, 50000],
        dtick=10000,
        title='X [km]'
    ),
    yaxis=dict(
        range=[-50000, 50000],
        dtick=10000,
        title='Y [km]',
    ),
    zaxis=dict(
        range=[-50000, 50000],
        dtick=10000,
        title='Z [km]'
    ),)
)

data = [
    ellipsoidPlot(reearth),

    go.Scatter3d(x=O_r_S__ecef[:,0],y=O_r_S__ecef[:,1],z=O_r_S__ecef[:,2],
                 name = 'orbit?',
                 marker=dict(
                     size=0,
                      color=0),
                 line = dict(color = ('red'), 
                             width = 1)
                )]


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [14]:
data = [
    dict(
        type="scattergeo",
        lon=lon_S.flatten()*180.0/pi, 
        lat=lat_S.flatten()*180.0/pi,
        mode='markers+lines'
        )]
iplot(data)

In [ ]:

# Plot groundtrack
figure(figNum+2);
clf;
# Draw earth outline map for reference.  'contour' command may work
# differently in older versions of MATLAB
contour(-180:179, -90:+89, topoplot, [0 0], 'blue');
hold on;
axis equal;
grid on;
set(gca,'XLim', [-180 +180], 'YLim', [-90 +90], ...
        'XTick', [-180:30:+180], 'Ytick', [-90:30:+90]);
title('Problem 3.3 - Ground Track');
xlabel('Longitude [deg]');
ylabel('Latitude [deg]');
# Use 'markers' instead of 'lines' for this plot to avoid distracting
# jumps in plot for data that crosses 180 [deg] longitude
plot(lon_S*180.0/pi, lat_S*180.0/pi, ...
     'Color', plotcolor, 'LineStyle', 'none', ...
     'Marker', 'o', 'MarkerFaceColor', plotcolor, 'MarkerSize', 2); 
plot(lon_S(1:ticki:end)*180.0/pi, lat_S(1:ticki:end)*180.0/pi, ...
     'Color', plotcolor, 'LineStyle', 'none', ...
     'Marker', 'o', 'MarkerFaceColor', plotcolor, 'MarkerSize', 5);



end